# Intelligent Realestate Investing 

In [91]:
#https://rapidapi.com/apidojo/api/realtor
import pandas as pd
import requests
import json #helps manipulate json
import csv # helps with csv files
from bs4 import BeautifulSoup
!pip install pyzillow

from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

You should consider upgrading via the 'c:\users\btindol\anaconda3\python.exe -m pip install --upgrade pip' command.


In [56]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [4]:
# Function using the realator api to grabe houses that are for sale!!

def api_property_list_for_sale(city, state, prop_type, limit=200):

    url = "https://realtor.p.rapidapi.com/properties/v2/list-for-sale"

    querystring = {"city":city,"limit":"200","offset":"0","state_code":state,"sort":"relevance","prop_type":prop_type}

    headers = {
        'x-rapidapi-key': "3098984b99msh9a120a8ade10e53p1ebdd1jsn7703d6e155d0",
        'x-rapidapi-host': "realtor.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    return response.json() # json format

In [5]:
# Once pick state and home type throw it through this function to get the proper readable format
def process_list_for_sale_response(response_json):
    """
    Process the list for sale API response.

    Convert each listing to a dataframe, append to a list, and concatenate to one dataframe.

    Parameters
    ----------
    @response_json [dictionary]: API response for list for sale

    Returns
    -------
    [dataframe] Dataframe of all list for sale responses

    """

    # empty dataframe
    dataframe_list = []

    # iterate through each for sale listing
    for l in response_json['properties']:

        # convert each listing to dataframe
        _temp_df = pd.DataFrame.from_dict(l, orient='index').T

        # append to dataframe list for all listings
        dataframe_list.append(_temp_df)

    # concatenate all dataframes, for missing col values enter null value
    return pd.concat(dataframe_list, axis=0, ignore_index=True, sort=False)

In [6]:
def make_house_data_clean(df_properties_for_sale_raw):
    df_properties_for_sale_raw = pd.DataFrame(df_properties_for_sale_raw)# transfer into dataframe
    df_pol_ps = df_properties_for_sale_raw['lot_size'].apply(pd.Series) # make this int column non interable into series then 
    df_properties_for_sale_raw['size'] = df_pol_ps['size'] # then make it into column 

    # pick out only the columns we want  and rename them 
    list2 = ['property_id', 'listing_id','address','prop_status','price', 'baths_full', 'baths', 'beds', 'building_size','last_update', 'lot_size','baths_half', 'size']
    result = df_properties_for_sale_raw[list2]
    result.columns = ['property_id', 'listing_id','address','prop_status','price', 'baths_full', 'baths', 'beds', 'building_size','last_update', 'lot_size','baths_half', 'lot_size']
    return result

In [7]:
def get_address_and_buildingsize(result):
    address = pd.json_normalize(result['address'])
    building_size = pd.json_normalize(result['building_size'])
    building_size.columns =['building_size','building_inside_units']
    result2 = pd.concat([address,building_size], axis=1)
    result3 = pd.concat([df,result2], axis=1)
    return result3


In [8]:
# Variables 
city = "myrtle beach"
state = "SC"
prop_type = "single_family"

property_list_for_sale_response = api_property_list_for_sale(
                                                             city=city, 
                                                             state=state, 
                                                             prop_type=prop_type,
                                                             limit=200)

In [9]:
df_properties_for_sale_raw = process_list_for_sale_response(response_json=property_list_for_sale_response)


In [10]:
df = make_house_data_clean(df_properties_for_sale_raw)
#list(df.columns)
df

,property_id,listing_id,address,prop_status,price,baths_full,baths,beds,building_size,last_update,lot_size,baths_half,lot_size
0,M9410361591,2929615304,"{'city': 'Myrtle Beach', 'line': '533 Ambrosia...",for_sale,235000,2,2,3,"{'size': 1449, 'units': 'sqft'}",2021-05-22T16:35:18Z,"{'size': 10890, 'units': 'sqft'}",NaN,10890.0
1,M9090577838,2929605773,"{'city': 'Myrtle Beach', 'line': '2185 Sanibel...",for_sale,424900,3,3,4,"{'size': 1725, 'units': 'sqft'}",2021-05-22T04:15:29Z,NaN,NaN,NaN
2,M9155203410,2929592057,"{'city': 'Myrtle Beach', 'line': '9035 Gatewic...",for_sale,249900,2,2,3,"{'size': 1394, 'units': 'sqft'}",2021-05-21T20:51:17Z,"{'size': 7841, 'units': 'sqft'}",NaN,7841.0
3,M9416395684,2929590343,"{'city': 'Myrtle Beach', 'line': '650 Elgin Ct...",for_sale,394900,3,4,4,"{'size': 2624, 'units': 'sqft'}",2021-05-22T00:31:43Z,"{'size': 15682, 'units': 'sqft'}",1,15682.0
4,M5723306696,2929587613,"{'city': 'Myrtle Beach', 'line': '2534 Goldfin...",for_sale,400000,2,2,3,"{'size': 2148, 'units': 'sqft'}",2021-05-21T19:44:11Z,"{'size': 13068, 'units': 'sqft'}",NaN,13068.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,M9453125981,2929083403,"{'city': 'Myrtle Beach', 'line': '1424 Brighto...",for_sale,255000,2,3,4,"{'size': 1850, 'units': 'sqft'}",2021-05-22T13:37:37Z,"{'size': 6098, 'units': 'sqft'}",1,6098.0
196,M9549522450,2929083084,"{'city': 'Myrtle Beach', 'line': '2653 Corn Pi...",for_sale,249000,2,2,3,"{'size': 1464, 'units': 'sqft'}",2021-05-10T21:58:38Z,"{'size': 19602, 'units': 'sqft'}",NaN,19602.0
197,M9698682059,2929082893,"{'city': 'Myrtle Beach', 'line': '6001-1454 S ...",for_sale,237000,3,3,4,"{'size': 1610, 'units': 'sqft'}",2021-05-17T15:11:36Z,"{'size': 1742, 'units': 'sqft'}",NaN,1742.0
198,M9517132030,2929080051,"{'city': 'Myrtle Beach', 'line': '3556 Lightho...",for_sale,198500,2,2,2,"{'size': 1076, 'units': 'sqft'}",2021-05-09T18:04:01Z,"{'size': 8712, 'units': 'sqft'}",NaN,8712.0


In [63]:
# expand the address and building size columns 
var2 = get_address_and_buildingsize(df)
var2['line']

0            533 Ambrosia Loop
1              2185 Sanibel Ct
2             9035 Gatewick Ct
3                 650 Elgin Ct
4            2534 Goldfinch Dr
                ...           
195          1424 Brighton Ave
196          2653 Corn Pile Rd
197    6001-1454 S Kings Hwy S
198        3556 Lighthouse Way
199       289 Wateree River Rd
Name: line, Length: 200, dtype: object

In [64]:
var2

,property_id,listing_id,address,prop_status,price,baths_full,baths,beds,building_size,last_update,...,county,fips_code,county_needed_for_uniq,lat,lon,neighborhood_name,time_zone,is_approximate,building_size,building_inside_units
0,M9410361591,2929615304,"{'city': 'Myrtle Beach', 'line': '533 Ambrosia...",for_sale,235000,2,2,3,"{'size': 1449, 'units': 'sqft'}",2021-05-22T16:35:18Z,...,Horry,45051,False,33.724152,-78.954019,Forestbrook,NaN,NaN,1449,sqft
1,M9090577838,2929605773,"{'city': 'Myrtle Beach', 'line': '2185 Sanibel...",for_sale,424900,3,3,4,"{'size': 1725, 'units': 'sqft'}",2021-05-22T04:15:29Z,...,Horry,45051,False,33.669617,-78.908846,South Myrtle Beach,America/New_York,NaN,1725,sqft
2,M9155203410,2929592057,"{'city': 'Myrtle Beach', 'line': '9035 Gatewic...",for_sale,249900,2,2,3,"{'size': 1394, 'units': 'sqft'}",2021-05-21T20:51:17Z,...,Horry,45051,False,33.749447,-78.939917,Carolina Forest,NaN,NaN,1394,sqft
3,M9416395684,2929590343,"{'city': 'Myrtle Beach', 'line': '650 Elgin Ct...",for_sale,394900,3,4,4,"{'size': 2624, 'units': 'sqft'}",2021-05-22T00:31:43Z,...,Horry,45051,False,33.741506,-78.909741,Carolina Forest,NaN,NaN,2624,sqft
4,M5723306696,2929587613,"{'city': 'Myrtle Beach', 'line': '2534 Goldfin...",for_sale,400000,2,2,3,"{'size': 2148, 'units': 'sqft'}",2021-05-21T19:44:11Z,...,Horry,45051,False,33.670222,-78.967307,NaN,America/New_York,NaN,2148,sqft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,M9453125981,2929083403,"{'city': 'Myrtle Beach', 'line': '1424 Brighto...",for_sale,255000,2,3,4,"{'size': 1850, 'units': 'sqft'}",2021-05-22T13:37:37Z,...,Horry,45051,False,33.635668,-79.045618,NaN,NaN,NaN,1850,sqft
196,M9549522450,2929083084,"{'city': 'Myrtle Beach', 'line': '2653 Corn Pi...",for_sale,249000,2,2,3,"{'size': 1464, 'units': 'sqft'}",2021-05-10T21:58:38Z,...,Horry,45051,False,33.723513,-78.988895,Forestbrook,NaN,NaN,1464,sqft
197,M9698682059,2929082893,"{'city': 'Myrtle Beach', 'line': '6001-1454 S ...",for_sale,237000,3,3,4,"{'size': 1610, 'units': 'sqft'}",2021-05-17T15:11:36Z,...,Horry,45051,False,33.626229,-78.960292,Ocean Lakes,NaN,NaN,1610,sqft
198,M9517132030,2929080051,"{'city': 'Myrtle Beach', 'line': '3556 Lightho...",for_sale,198500,2,2,2,"{'size': 1076, 'units': 'sqft'}",2021-05-09T18:04:01Z,...,Horry,45051,False,33.727377,-78.876152,Myrtlewood,America/New_York,NaN,1076,sqft


In [18]:
list(var2.columns)

['property_id',
 'listing_id',
 'address',
 'prop_status',
 'price',
 'baths_full',
 'baths',
 'beds',
 'building_size',
 'last_update',
 'lot_size',
 'baths_half',
 'lot_size',
 'city',
 'line',
 'postal_code',
 'state_code',
 'state',
 'county',
 'fips_code',
 'county_needed_for_uniq',
 'lat',
 'lon',
 'neighborhood_name',
 'time_zone',
 'is_approximate',
 'building_size',
 'building_inside_units']

# Now remove all of the numbers for the address and get zestimate and put them in to melissa .com for the other addresses on the street
https://www.melissa.com/v2/lookups/addresssearch/?number=&street=Carino+Ct&city=Paw+Paw&state=MI&zip=49079-9661&freeForm=

# Melisa.com is giving me problems try to use selenium and click and sign in to grab street.. 

In [65]:
# Funciton that thakes the dataframe from realtor.com and then throws it through melissa.com and gets all of the neighbors..
# Might have to include this intoa loop instead of a column.. 
def get_street_neighbors(var2):
    # get the street column for that index in the 
    street = var2['line'][0]
    result = ''.join([i for i in street if not i.isdigit()]) # search for digits and replace them when searching for the street
    result =result.strip() # remove leading and trailing spaces
    street = result.replace(" ", "+") # replace the space betwee with plus. 
    
    # get state
    state = var2['state'][0]
    state = us_state_abbrev[state] # make it the abbriviation not the name
    
    # get zip code
    zipcode = var2['postal_code'][0]

    # Get the city 
    city = var2['city'][0]
    city = city.replace(" ", "+") # replace space with plus
    
    # Grab the url with the formatted string
    link_to_neighbors = 'https://www.melissa.com/v2/lookups/addresssearch/?number=&street={0}&city={1}&state={2}&zip={3}&freeForm='.format(street,city,state,zipcode)
    
    # now grab the html the one and ony table on that page on mellisa.com
    indicators = pd.read_html(link_to_neighbors)
    variabledf = indicators[0] # brab the table... 
    
    return variabledf

In [74]:
final = get_street_neighbors(var2)
final

In [67]:
street = var2['line'][0]
result = ''.join([i for i in street if not i.isdigit()]) # search for digits and replace them when searching for the street
result =result.strip() # remove leading and trailing spaces
street = result.replace(" ", "+")

state = var2['state'][0]
state = us_state_abbrev[state]
zipcode = var2['postal_code'][0]

city = var2['city'][0]
city = city.replace(" ", "+")
state

'SC'

In [68]:
link_to_neighbors = 'https://www.melissa.com/v2/lookups/addresssearch/?number=&street={0}&city={1}&state={2}&zip={3}&freeForm='.format(street,city,state,zipcode)
link_to_neighbors

'https://www.melissa.com/v2/lookups/addresssearch/?number=&street=Ambrosia+Loop&city=Myrtle+Beach&state=SC&zip=29579&freeForm='

In [69]:
print(link_to_neighbors)

https://www.melissa.com/v2/lookups/addresssearch/?number=&street=Ambrosia+Loop&city=Myrtle+Beach&state=SC&zip=29579&freeForm=


In [79]:
indicators = pd.read_html(link_to_neighbors)
#variabledf = indicators[0] # grab the first element of the html which is the actual table
#variabledf
indicators

# Try to get mellissa.com information 

In [106]:

#object of ChromeOptions class
c = webdriver.ChromeOptions()
#incognito parameter passed
#c.add_argument("--incognito")
#set chromodriver.exe path
#driver = webdriver.Chrome('C:/Users/btindol/OneDrive - Stryker/ChromeDriver/chromedriver.exe')

driver = webdriver.Chrome(executable_path='C:/Users/btindol/OneDrive - Stryker/ChromeDriver/chromedriver.exe')#,options=c)
driver.implicitly_wait(0.5)
#launch URL
driver.get(link_to_neighbors)

WebDriverWait(driver, 10).until(EC.alert_is_present())
driver.switch_to.alert.accept()
email = 'blaketindol@gmail.com'
passwordz = You know waht it is 
driver.implicitly_wait(3)
emailfield= driver.find_element_by_id("ctl00_ContentPlaceHolder1_Signin1_txtEmail")
passwordmessagefield = driver.find_element_by_id("ctl00_ContentPlaceHolder1_Signin1_txtPassword")
emailfield.send_keys(email)
passwordmessagefield.send_keys(passwordz)
submitbutton = driver.find_element_by_id("ctl00_ContentPlaceHolder1_Signin1_btnLogin")
submitbutton.click()
#link_to_neighbors = 'https://www.melissa.com/v2/lookups/addresssearch/?number=&street={0}&city={1}&state={2}&zip={3}&freeForm='.format(street,city,state,zipcode)
#indicators = pd.read_html(link_to_neighbors)
#variabledf = indicators[0] # grab the first element of the html which is the actual table
#variabledf
#indicators
excelfile = driver.find_element_by_xpath('//*[@id="content"]/section[3]/div/div/div/span/div/div[3]/input[3]')
excelfile.click()

In [ ]:
excelfile = driver.find_element_by_xpath('//*[@id="content"]/section[3]/div/div/div/span/div/div[3]/input[3]')
excelfile.click()

In [103]:
passwrd = 'blaketindol@gmail.com'
passwordmessagefield = driver.find_element_by_id("ctl00_ContentPlaceHolder1_Signin1_txtEmail")
passwordmessagefield.send_keys(passwrd)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=90.0.4430.212)


In [ ]:
ctl00_ContentPlaceHolder1_Signin1_txtEmail

In [81]:
df = pd.read_csv("address.csv")
df

,HouseNumber,Street,StreetSuffix,City,State,Zip,StreetName,Address,StateName,Zip4,AddressType,RBDI,Active,Fips,County,ResultCode,MAK
0,500,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,500 Ambrosia Loop,South Carolina,NaN,,Residential,Yes,45051,horry,AS03,1442484286
1,501,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,501 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051,horry,AS01,1078919714
2,505,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,505 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051,horry,AS01,6241258760
3,509,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,509 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051,horry,AS01,9622347032
4,513,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,513 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051,horry,AS01,1328098783
5,517,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,517 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051,horry,AS01,2245176586
6,520,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,520 Ambrosia Loop,South Carolina,7047.0,Street,Residential,Yes,45051,horry,"AS01,AS17",4268283008
7,521,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,521 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051,horry,AS01,4266960723
8,525,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,525 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051,horry,AS01,5136901644
9,529,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,529 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051,horry,AS01,1343835818


# Use zillow to get data

In [83]:
zillow_data = ZillowWrapper("X1-ZWz1fjckjdd8gb_a2eph")

In [86]:
# Grab one of the hosue numbers on the street
address = df['Address'][0]
zipcode = df['Zip'][0]
address

'500 Ambrosia Loop'

In [87]:
deep_search_response = zillow_data.get_deep_search_results(address, zipcode, True)
result = GetDeepSearchResults(deep_search_response)
print("Bathrooms " +result.bathrooms)
print("Bedrooms " + result.bedrooms)
print("County fips " + result.fips_county)
print(result.total_rooms)
print("The Zillow ID is " +result.zillow_id)
print("zestimate percentile is " +result.zestimate_percentile)
print("zestimate last updated " +result.zestimate_last_updated)
print("zestimate amount  " +result.zestimate_amount)
print(result.map_this_home_link)
print("Valuation Range High End " +result.zestimate_valuation_range_high)
print("Valuation Range Low End " +result.zestimate_valuation_range_low)
print("Last sold price " +result.last_sold_price)
print("City " + result.city)
print(result.graph_data_link)
print("Rent Zestimate Amount " + result.rentzestimate_amount)
print("Zipcode " + result.zipcode)
print("Street " + result.street)

Bathrooms 2.5
Bedrooms 3
County fips 45051
None
The Zillow ID is 92415525
zestimate percentile is 0
zestimate last updated 05/22/2021
zestimate amount  286541
http://www.zillow.com/homes/92415525_zpid/
Valuation Range High End 300868
Valuation Range Low End 269349
Last sold price 252781
City Myrtle Beach
http://www.zillow.com/homedetails/500-Ambrosia-Loop-Myrtle-Beach-SC-29579/92415525_zpid/#charts-and-data
Rent Zestimate Amount 1750
Zipcode 29579
Street 500 Ambrosia Loop
